In [14]:
# --- 1. Importar Librerías Esenciales ---

# Librerías para manipulación de datos
import pandas as pd
# Uso: Fundamental para manejar DataFrames (tus tablas de datos).
#      Se usará para cargar los CSV, manipular columnas, etc.

import numpy as np
# Uso: Para operaciones numéricas eficientes, especialmente con arrays de números.
#      Scikit-learn trabaja mucho con arrays de NumPy internamente.

import re
# Uso: Para expresiones regulares. Necesario para la función de extracción de 'Title' del nombre
#      y también para extraer el 'Ticket_Prefix' del número de billete.

# Módulos de Scikit-learn para la construcción del Pipeline y Modelado
from sklearn.model_selection import train_test_split, GridSearchCV
# train_test_split: Para dividir el dataset en conjuntos de entrenamiento y validación.
# Uso: Asegura que el modelo se evalúe en datos no vistos durante el entrenamiento.
# GridSearchCV: Para la optimización de hiperparámetros de nuestro modelo y/o pipeline.
# Uso: Busca sistemáticamente la mejor combinación de parámetros.

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
# StandardScaler: Para estandarizar (escalar) las características numéricas.
# Uso: Transforma los datos para que tengan media 0 y desviación estándar 1, útil para muchos algoritmos.
# OneHotEncoder: Para convertir características categóricas (como 'Sex', 'Embarked') a formato numérico binario.
# Uso: Permite que los modelos de ML trabajen con datos categóricos.
# FunctionTransformer: Para integrar funciones personalizadas (como tus funciones de ingeniería de características)
# Uso: Te permite usar tus funciones Python dentro del pipeline de Scikit-learn.

from sklearn.impute import SimpleImputer
# Uso: Para manejar los valores faltantes. Por ejemplo, rellenar 'Age' o 'Fare' con la media.

from sklearn.compose import ColumnTransformer
# Uso: Permite aplicar diferentes transformaciones a diferentes columnas del DataFrame.
#      Es clave para nuestro pipeline, ya que tenemos columnas numéricas y categóricas.

from sklearn.pipeline import Pipeline
# Uso: La clase principal para construir tu pipeline. Encadena todos los pasos de preprocesamiento y el modelo.
#      Esto garantiza que todas las transformaciones se apliquen consistentemente.

from sklearn.ensemble import RandomForestClassifier
# Uso: El algoritmo de Machine Learning que usaremos para nuestro modelo de clasificación binaria.
#      Es un clasificador de conjunto robusto y de alto rendimiento.

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Uso: Para evaluar el rendimiento de nuestro modelo.
# accuracy_score: La métrica principal para el concurso de Kaggle.
# confusion_matrix: Muestra la cantidad de verdaderos positivos, negativos, falsos positivos y negativos.
# classification_report: Proporciona precisión (precision), recall (sensibilidad) y F1-score por clase.

# Librerías para visualización de datos (aunque en este notebook será más para mostrar resultados)
import matplotlib.pyplot as plt
import seaborn as sns
# Uso: Para crear gráficos, como la matriz de confusión, para visualizar y entender los resultados del modelo.

# Configuración para gráficos
sns.set_style("whitegrid")
plt.style.use("seaborn-v0_8-darkgrid")
# Uso: Ajustes estéticos para que los gráficos se vean bien.

In [15]:
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-darkgrid')

In [16]:
df_train_raw = pd.read_csv('../data/raw/train.csv')
df_test_raw = pd.read_csv('../data/raw/test.csv')
df_join_raw = pd.concat([df_train_raw.drop('Survived', axis=1), df_test_raw], ignore_index=True)

print(f'Dimensiones iniciales: {df_join_raw.shape}')
print(f'Columnas iniciales: {df_join_raw.columns.to_list}')

Dimensiones iniciales: (1309, 11)
Columnas iniciales: <bound method IndexOpsMixin.tolist of Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')>


In [17]:
df_join_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [18]:

def get_title_from_name(df):
    df = df.copy()
    def extract_title(name):
        title_extracted = re.search(' ([A-Za-z]+).', name)
        if title_extracted:
            return title_extracted.group(1)
        return 'rare'
    df['Title'] = df['Name'].apply(extract_title)
    df = df.drop('Name', axis=1)
    return df


def get_deck_from_cabin(df):
    df = df.copy()
    df['Cabin'] = df['Cabin'].fillna('Unknown')
    df['Deck'] = df['Cabin'].transform(lambda x: x[0])
    df = df.drop('Cabin', axis=1)
    return df

def get_agency_ticket_numbers_from_ticket(df):
    df = df.copy()
    def extract_prefix(ticket):
        prefix_extracted = re.match(r'([A-Za-z\./]+)', ticket)
        if prefix_extracted:
            return prefix_extracted.group(1).replace('.', '').replace('/', '').upper()
        return 'NO_AGENCY'
    df['Agency'] = df['Ticket'].apply(extract_prefix)
    ticket_counts = df['Ticket'].value_counts()
    df['Ticket_number'] = df['Ticket'].map(ticket_counts)
    df = df.drop('Ticket', axis=1)
    return df


feature_engineering_pipeline = Pipeline([
    ('get_title', FunctionTransformer(get_title_from_name, validate=False)),
    ('get_deck', FunctionTransformer(get_deck_from_cabin, validate=False)),
    ('get_agency', FunctionTransformer(get_agency_ticket_numbers_from_ticket, validate=False))
])
temp_df = df_join_raw.copy()
res_temp_df = feature_engineering_pipeline.fit_transform(temp_df)
res_temp_df
#print('Cargado transformadores de ingenieria de features')

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,Agency,Ticket_number
0,1,3,male,22.0,1,0,7.2500,S,Mr,U,A,1
1,2,1,female,38.0,1,0,71.2833,C,Mrs,C,PC,2
2,3,3,female,26.0,0,0,7.9250,S,Miss,U,STONO,1
3,4,1,female,35.0,1,0,53.1000,S,Mrs,C,NO_AGENCY,2
4,5,3,male,35.0,0,0,8.0500,S,Mr,U,NO_AGENCY,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,male,NaN,0,0,8.0500,S,Mr,U,A,1
1305,1306,1,female,39.0,0,0,108.9000,C,y,C,PC,3
1306,1307,3,male,38.5,0,0,7.2500,S,Mr,U,SOTONOQ,1
1307,1308,3,male,NaN,0,0,8.0500,S,Mr,U,NO_AGENCY,1


In [19]:
def reorgize_ticket_class(df):
    df = df.copy()
    def condition_class(ticket_class):
        class_mapping = { 1: 2,  2: 1, 3: 0}
        return class_mapping.get(ticket_class, 0)
    df['Pclass'] = df['Pclass'].apply(condition_class)
    return df

numeric_transformer_multivariate = ['Age']
numeric_transformer = Pipeline(steps=[
    ('reorganize', FunctionTransformer(reorgize_ticket_class, validate=False))
])
res_temp2 = numeric_transformer.fit_transform(res_temp_df)
res_temp2
#print('Cargado imputadores numericos y categoricos')

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,Agency,Ticket_number
0,1,0,male,22.0,1,0,7.2500,S,Mr,U,A,1
1,2,2,female,38.0,1,0,71.2833,C,Mrs,C,PC,2
2,3,0,female,26.0,0,0,7.9250,S,Miss,U,STONO,1
3,4,2,female,35.0,1,0,53.1000,S,Mrs,C,NO_AGENCY,2
4,5,0,male,35.0,0,0,8.0500,S,Mr,U,NO_AGENCY,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,male,NaN,0,0,8.0500,S,Mr,U,A,1
1305,1306,2,female,39.0,0,0,108.9000,C,y,C,PC,3
1306,1307,0,male,38.5,0,0,7.2500,S,Mr,U,SOTONOQ,1
1307,1308,0,male,NaN,0,0,8.0500,S,Mr,U,NO_AGENCY,1
